# A class allowing to easily handle and create folium map is defined here

In [48]:
import re
import json
import folium
import fileinput
import numpy as np
import pandas as pd
import seaborn as sns
import branca.colormap as cm
from datetime import datetime
from selenium import webdriver
from BindColorMap import BindColormap
from IPython.core.display import display
from folium.plugins import TimestampedGeoJson

In [1]:
"""
Parameters:
-----------

Returns:
-----------

"""

"""
GeoMap is a class allowing an easy creation of multi-layer folium maps

Attributes:
-----------

Methods:
-----------

"""

'\nGeoMap is a class allowing an easy creation of multi-layer folium maps\n\nAttributes:\n-----------\n\nMethods:\n-----------\n\n'

In [2]:
def merge_transport_df(file_lines, file_systems, file_stations, file_station_lines, file_cities):
    """merge the transportation data set into an unique summary dataframe
    Parameters:
    -----------
    file_lines : string
        full path to the lines csv file
    file_systems : string
        full path to the systems csv file
    file_stations : string
        full path to the stations csv file
    file_station_lines : string
        full path to the station_lines csv file
    file_cities : string
        full path to the cities csv file
    
    Returns:
    -----------
    transports_df : pandas DataFrame
        the merged DataFrame
    
    """
    
    # 1) load dataframes
    cities = pd.read_csv(file_cities)
    lines = pd.read_csv(file_lines).rename(columns = {'id':'line_id', 'name':'line_name', 'url_name':'line_url_name', 'color':'line_color'})
    systems = pd.read_csv(file_systems).rename(columns = {'id':'system_id', 'name':'system_name'}).drop('city_id', axis = 1)
    stations = pd.read_csv(file_stations).rename(columns = {'id':'station_id', 'name':'station_name', 'geometry':'station_geometry',
                                                   'buildstart':'station_buildstart', 'opening':'station_opening',
                                                   'closure':'station_closure'}).drop('city_id', axis = 1)
    station_lines = pd.read_csv(file_station_lines).rename(columns = {'id':'station_lines_id'}).drop(['city_id', 
                                                   'created_at', 'updated_at'], axis = 1)
    
    
    # 2) Fix duplicated city names by adding the state to the city name
    dupl_cities = cities[cities.duplicated(['url_name'], keep = False)]
    for i in  dupl_cities.index.values:
        cities.loc[i, 'url_name'] = cities.loc[i, 'url_name'] + '_' + cities.loc[i, 'country_state']
    cities = cities[['id', 'url_name']].rename(columns = {'id':'city_id', 'url_name':'city_name'})
    
    # 3) Merge dataframes
    lines = pd.merge(lines, cities, on = ['city_id'], how = 'outer')
    tmp1 = pd.merge(lines, systems, on = ['system_id'], how = 'outer')
    tmp2 =  pd.merge(station_lines, stations, on = ['station_id'], how = 'outer')
    transports_df = pd.merge(tmp1, tmp2, on = 'line_id', how = 'outer')
    
    return transports_df

In [3]:
class GeoMap():
    """
    GeoMap is a class allowing an easy creation of multi-layer folium maps
    
    Attributes:
    -----------
    geo_map : folium object
    
    Methods:
    -----------
    map_to_color(color_palette, list_of_values, min_range = None, max_range = None)
        maps a list of values to a color defined by a color palette, bounding the max and min colors to specified values
        
    generate_gradient_dict(color_palette)
        returns an dictionary of same length of color_palette where each color is assigned to a fraction, to be used in HeatMap layers
    
    geojson_layer(layer_name, jsonFile, displayColorData = None)
        returns a layer where geographical data contained in the input geojson file are displayed
        
    pandas_layer(df, layer_name, displayType = 'Density', densityDataName = None)
        returns a layer where data (geographical points) contained in a pandas DataFrame are displayed with different styles: Density or Circles or Points
        
    icon_layer(df, layer_name)
        returns a layer where data contained in a pandas DataFrame are displayed as specific icons fetched from the fontawesome list
        
    add_layer_list(layer_list)
        adds the input layer to the GeoMap attribute
        
    add_tiles(tiles_list = ['OpenStreetMap', 'cartodbdark_matter'], tiles_names = ['Street Map', 'Dark Map'])
        adds tiles ("view modes") to the GeoMap attribute
        
    add_mini_map()
        adds a mini-map representation ("zoom-out loclaization window") to the GeoMap attribute
    
    get_map()
        returns the GeoMap attribute
        
    save_map(saving_destination_path, file_name)
        saves the GeoMap attribute as html file
        
    get_non_null_rows(df, column_check)
        returns rows containing valid numerical data only (NaNs are excluded) in the input pandas DataFrame
    """
    
    def __init__(self, coord_start = [46.519164, 6.566719]):
        """Class constructor
        
        Parameters:
        -----------
        coord_start : array
            the coordinates on which the folium map is initially centered. coord_start[latitude, longitude]
            default: EPFL coordinates
        """
        self.geo_map = folium.Map(location = coord_start, control_scale = True, zoom_start = 11)
    
    def map_to_color(self, color_palette, list_of_values, min_range = None, max_range = None):
        """map a list of values to discrete colors
        If the min_range is specified, any value smaller than this value will be assigned to the minimal color. Similarly for max_range.
        
        Parameters:
        -----------
        color_palette  : array of hex-encoded colors
        list_of_values : array
            the list of values that have to be mapped to a color
        min_range      : float
            the minimal value associated to the minimum color (default = None)
        max_range      : float
            the maximal value associated to the minimum color (default = None)
        
        Returns:
        -----------
        mapped_colors : array
            the list of colors mapped to the input values
        """
        
        if min_range == None:
            min_range = np.min(list_of_values)
        if max_range == None:
            max_range = np.max(list_of_values)
        
        intervals = np.linspace(min_range, max_range, len(color_palette)+1)

        mapped_colors = []
        for i in list_of_values:
            # we have N-1 colors
            for c in range (0, len(intervals) - 1):
                if c == 0 and i < intervals[c]:
                    mapped_colors = np.append(mapped_colors, color_palette[c])
                if c < len(intervals) and i >= intervals[c] and i < intervals[c + 1]:
                    mapped_colors = np.append(mapped_colors, color_palette[c])
                    break
                elif c == len(intervals) - 2 and i >= intervals[c]:
                    mapped_colors = np.append(mapped_colors, color_palette[c])
        return mapped_colors
    
    def generate_gradient_dict(self, color_palette):
        """generate a fraction-linked gradient of colors
        
        If for example the input contains N colors, an homogeneous division of the [0, 1] interval will be 
        created and the center of each interval is assigned to each of the N intervals centers.To be used
        with folium HeatMap
        
        Parameters:
        -----------
        color_palette  : array of hex-encoded colors
        
        Returns:
        -----------
        gradient : array
        """
        
        intervals = np.linspace(0, 1, len(color_palette)+1)
        
        # compute center of each interval:
        centers = np.round((intervals[1:] + intervals[:-1]) / 2, 3)
        
        gradient = {}
        for c in range(0, len(color_palette)):
            gradient.update({centers[c] : color_palette[c]})
        return gradient
    
    # displayColorData: a vector of values
    def geojson_layer(self, layer_name, jsonFile, displayColorData = None):
        """generate a layer displaying geojson graphical informations
        
        Parameters:
        -----------
        layer_name : string
            the name of the layer
        jsonFile : opened geojson file
            the file containing the geographical informations
        displayColorData : bool
            if true, display data with a color gradient
        
        Returns:
        -----------
        [layer_gjson, colormap] = array
            layer_gjson is the created geojson layer
            colormap is a branca colormap. Created only if displayColorData is specified, otherwise colormap = None
        """
        
        layer_gjson = folium.FeatureGroup(name = layer_name)
        
        if displayColorData != None:
            max_value_color = np.quantile(displayColorData.to_list, 0.95)
            min_value_color = np.quantile(displayColorData.to_list, 0.05)
            color_palette = sns.color_palette('RdYlGn').as_hex()
            colors = self.map_to_color(color_palette, displayColorData.to_list(), min_range = min_value_color, max_range = max_value_color)
            colormap = cm.StepColormap(colors = color_palette, vmin=min_value_color, vmax=max_value_color,
                                                      caption = layer_name)
            
        print('\n')
        for i, geo_json in enumerate(jsonFile['features']):
            print('geojson_layer construction running... ' + str(int(np.ceil((i/len(jsonFile['features']))*100))) + '%', end='\r')
            
            if displayColorData != None:
                fill_col = colors[i]
                edges_color = 'white'
            else:
                fill_col = 'yellow'
                edges_color = 'blue'
            lj = folium.GeoJson(
                geo_json,
                name='geojson',
                style_function=lambda feature: {
                    'fillColor': fill_col,
                    'color' : edges_color,
                    'weight' : 1,
                    'fillOpacity' : 0.2,
                    }
                )
            popup = folium.Popup(geo_json['properties']['neighbourhood'])
            lj.add_child(popup)
            lj.add_to(layer_gjson)
            #airbnb_map.add_child(gj)
        
        if displayColorData != None:
            return [layer_gjson, colormap]
        else:
            return [layer_gjson, None]
        
        #layer_gjson.add_to(airbnb_map)
        #folium.LayerControl().add_to(airbnb_map)
        
    
    # displayType can be: Density, Points, Circles
    def pandas_layer(self, df, layer_name, displayType = 'Density', densityDataName = None):
        """generate a layer displaying the content of the input DataFrame
        
        Parameters:
        -----------
        df : pandas DataFrame object
            df must contain columns: latitude, longitude, densityDataName
        layer_name : string
            the name of the layer
        displayType : string
            the type of visualization of the input data. Can be: 'Density', 'Points', 'Circles'. Default: 'Density'
        densityDataName : the column in df containing the color weights for each points if a colorscale want to be used to visualize data. Default: None
        
        Returns:
        -----------
        [layer_pandas, colormap]
        layer_pandas is the created  layer
        colormap is a branca colormap. Created only if densityDataName is specified, otherwise colormap = None
        """
        
        layer_pandas = folium.FeatureGroup(name = layer_name);
        
        if densityDataName != None:
            max_value = np.max(df[densityDataName])
            max_value_color = np.quantile(df[densityDataName].tolist(), 0.95)
            min_value_color = np.quantile(df[densityDataName].tolist(), 0.05)
            color_palette = sns.color_palette('RdYlGn').as_hex()
            colors = self.map_to_color(color_palette, df[densityDataName].tolist(), min_range = min_value_color, max_range = max_value_color)
            colormap = cm.StepColormap(colors = color_palette, vmin=min_value_color, vmax=max_value_color,
                                                      caption = layer_name)
        
        print('\n')
        if displayType in ['Points', 'Circles']:
            for index, row in df.iterrows():
                print('pandas_layer construction  running... ' + str(int(np.ceil((index/df.shape[0])*100))) + '%', end='\r')

                if displayType == 'Points' and densityDataName == None:
                    lp = folium.CircleMarker([row['latitude'], row['longitude']],
                                            radius = 2,
                                            color = 'red',
                                            fill_color = 'red')
                    lp.add_to(layer_pandas)
                elif displayType == 'Points' and densityDataName != None:
                    lp = folium.CircleMarker([row['latitude'], row['longitude']],
                                            radius = 2,
                                            color = colors[index-1],
                                            fill_color = str(colors[index]))
                    lp.add_to(layer_pandas)
                elif displayType == 'Circles' and densityDataName != None:
                    lp = folium.CircleMarker([row['latitude'], row['longitude']],
                                            radius = 20 * row[densityDataName]/max_value,
                                            color = colors[index],
                                            fill_color = colors[index])
                    lp.add_to(layer_pandas)
                    
        elif (displayType == 'Density') and (densityDataName != None):
            print('pandas_layer construction  running... ')
            
            points = df[['latitude', 'longitude', densityDataName]]
            points = points.loc[(df[densityDataName] >= min_value_color) & (df[densityDataName] <= max_value_color), :].values
            lp = folium.plugins.HeatMap(points, gradient = self.generate_gradient_dict(color_palette))
            print(self.generate_gradient_dict(color_palette))
            lp.add_to(layer_pandas)
        
        if densityDataName != None:
            return [layer_pandas, colormap]
        else:
            return [layer_pandas, None]
    
    # df columns must contain: [latitude, longitude, icon_type] where the icon_type must be an icon name 
    # available here: https://fontawesome.com/icons?d=listing. df can eventually contain the color of the icons: icon_color
    def icon_layer(self, df, layer_name):
        """generate a layer displaying the input data as icons
        
        
        Parameters:
        -----------
        df : pandas DataFrame object
            df must contain columns: latitude, longitude, icon_type
            the icon_type must be an icon name available here: https://fontawesome.com/icons?d=listing
            df can eventually contain a columns specifying the color of the icons: icon_color
        
        Returns:
        -----------
        [icon_layer, None]
            icon_layer is the created  layer
        
        """
        
        layer_icons = folium.FeatureGroup(name = layer_name);
        if 'icon_color' in df.columns:
            colors = df.icon_color.to_list()
        else:
            colors = ['cadetblue']*df.shape[0]
        
        for index, row in df.iterrows():
            # use prefix fa for fontawesome icons
            li = folium.Marker([row['latitude'], row['longitude']],
                              icon = folium.Icon(icon = row[icon_type],
                              prefix = 'fa'), color = colors[index])
            li.add_to(layer_icons)
        
        return [icon_layer, None]
    
        
    # layer_list is a list of [layer, color_map] or [layer, None] if the colormap is not present
    def add_layer_list(self, layer_list):
        """add a list of input folium layers to the GeoMap attribute
        
        If a colormap is present, it will be linked to the corresponding layer
        
        Parameters:
        -----------
        layer_list : array 
            array of [folium layer, branca color map]
        
        Returns:
        -----------
        -
        """
        
        for layer, cmap in layer_list:
            if cmap != None:
                self.geo_map.add_child(layer)
                self.geo_map.add_child(cmap)
                self.geo_map.add_child(BindColormap(layer, cmap))
            else:
                self.geo_map.add_child(layer)
        folium.LayerControl('topleft').add_to(self.geo_map)
            
    def add_tiles(self, tiles_list = ['OpenStreetMap', 'cartodbdark_matter'], tiles_names = ['Street Map', 'Dark Map']):
        """add different tiles ("Visualization modes") to the GeoMap attribute
        
        Parameters:
        -----------
        tiles_list : array
            array of folium tiles names. Default: tiles_list = ['OpenStreetMap', 'cartodbdark_matter']
        tiles_names : array
            array of names used as layer names for the added tiles. Default: tiles_names = ['Street Map', 'Dark Map']
            
        Returns:
        -----------
        -
        """
        
        # remove the default tile:
        del self.geo_map._children['openstreetmap']
        count = 0
        for tile, name_ in zip(tiles_list, tiles_names):
            # show first added tile when open the map
            if count == 0:
                lt = folium.TileLayer(tiles = tile, name = name_, show = True).add_to(self.geo_map)
            else:
                lt = folium.TileLayer(tiles = tile, name = name_).add_to(self.geo_map)
            count += 1
            
    def add_mini_map(self):
        """add a mini-map folium object to the GeoMap attribute
        
        Parameters:
        -----------
        -
        
        Returns:
        -----------
        -
        """
        
        minimap = folium.plugins.MiniMap()
        self.geo_map.add_child(minimap)
    
    def get_map(self):
        """return the GeoMap attribute
        
        Parameters:
        -----------
        -
        
        Returns:
        -----------
        self.geo_map : folium object
        
        """
        return self.geo_map
    
    def save_map(self, saving_destination_path, file_name):
        """save the GeoMap attribute as html file
        
        Parameters:
        -----------
        saving_destination_path : string
            the path of the saving destination folder
        file_name : string
            the name of the saved file
        
        Returns:
        -----------
        -
        """
        self.geo_map.save(saving_destination_path + '/' + file_name + '.html')
        
    def get_non_null_rows(self, df, column_check):
        """returns the non-null rows of the input pandas DataFrame containing only valid numerical values for a certain column (NaNs are excluded)
        
        Parameters:
        -----------
        df : pandas DataFrame
        column_check : string
            name of the df column that is tested
        
        Returns:
        -----------
        filtered df : pandas DataFrame
        
        """
        return df[np.isfinite(df[column_check])].reset_index()
            

## TODO list:

In [ ]:
# TODO: 
# check circles sizing system
# check density: should sample the data otherwise too slow in execution and too large output files -> too slow in display
# try to pass al points at once as a list, with all the colors in a list to avoid for loops: coord = (dictionnaire[support].lat,dictionnaire[support].lon)
# use folium.features.Choropleth to display neighborhood: advantages such as display names when mouse overing




## Example of MapClass code usage

In [372]:
input_csv_file = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/DataSet/2019-09-14_Amsterdam_listings_detailed.csv'
geojson_file   = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/DataSet/NaT_Amsterdam_neighbourhoods.geojson'
saving_path    = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/Outputs'

df = pd.read_csv(input_csv_file, low_memory = False);
coord_start = [df.latitude.mean(), df.longitude.mean()]

with open(geojson_file) as f:
    jsonFile = json.load(f)

columns = ['review_scores_rating', 'review_scores_cleanliness', 'review_scores_checkin',
           'review_scores_communication', 'review_scores_location', 'review_scores_value']
columns_names = [w.replace('_', ' ') for w in columns]

# non_null = df[np.isfinite(df['review_scores_rating'])].reset_index()
# non_null = non_null.loc[0:1000, :]

In [397]:
cart = GeoMap(coord_start);
cart.add_tiles()
layers = [cart.geojson_layer('Neighborhood', jsonFile, displayColorData = None)]
for col, col_name in zip(columns, columns_names):
    non_null = df[np.isfinite(df[col])].sample(n = 1000).reset_index()
    layers += [cart.pandas_layer(non_null, col_name, displayType = 'Points', densityDataName = col)]
    
cart.add_mini_map()
cart.add_layer_list(layers)

output = cart.get_map()
cart.save_map(saving_path, 'test')
output




geojson_layer construction running... 96%

pandas_layer construction  running... 100%

pandas_layer construction  running... 100%

pandas_layer construction  running... 100%

pandas_layer construction  running... 100%

pandas_layer construction  running... 100%



## Example of merge_transport_df code usage

In [307]:
file_lines         = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/Transports/lines.csv'
file_systems       = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/Transports/systems.csv'
file_stations      = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/Transports/stations.csv'
file_station_lines = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/Transports/station_lines.csv'
file_cities        = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/Transports/cities.csv'


transports_df = merge_transport_df(file_lines, file_systems, file_stations, file_station_lines, file_cities)
# drop closed transportation systems if info exist
transports_df = transports_df.loc[(transports_df.station_closure > 2019) | (transports_df.station_closure.isna())]

transports_amsterdam = transports_df[transports_df.city_name == 'london']
transports_amsterdam.head()


,line_id,city_id,line_name,line_url_name,line_color,system_id,transport_mode_id,city_name,system_name,station_lines_id,station_id,station_name,station_geometry,station_buildstart,station_opening,station_closure
5178,118.0,69.0,Waterloo & City Line,118-waterloo-&-city-line,#50e3c2,259.0,4.0,london,London Underground,3712.0,782.0,\n\nBank,POINT(-0.0898129050862336 51.5132922457233),1894.0,1898.0,999999.0
5179,118.0,69.0,Waterloo & City Line,118-waterloo-&-city-line,#50e3c2,259.0,4.0,london,London Underground,3739.0,783.0,NaN,POINT(-0.113046029341803 51.502667678402),1894.0,1898.0,999999.0
5180,115.0,69.0,Northern Line,115-norther-line,#000,259.0,4.0,london,London Underground,3705.0,784.0,NaN,POINT(-0.113300125722532 51.5025623884227),1923.0,1926.0,999999.0
5181,115.0,69.0,Northern Line,115-norther-line,#000,259.0,4.0,london,London Underground,3826.0,842.0,NaN,POINT(-0.147704295554433 51.4528010475682),1923.0,1926.0,999999.0
5182,115.0,69.0,Northern Line,115-norther-line,#000,259.0,4.0,london,London Underground,3831.0,843.0,NaN,POINT(-0.151918266139774 51.4447032610585),1923.0,1926.0,999999.0


In [ ]:
display(cities.head())
display(lines.head())  
display(systems.head())
display(stations.head())
display(station_lines.head())